In [0]:
import torch
import numpy as np

In [0]:
input_size = 4 # h,e,l,o(4개의 차원)
hidden_size = 2

In [0]:
h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]

input_data_np = np.array([[h,e,l,l,o],
                         [e,o,l,l,l],
                         [l,l,e,e,l]], dtype = np.float32)

input_data = torch.Tensor(input_data_np)
rnn = torch.nn.RNN(input_size, hidden_size)
outputs, _status = rnn(input_data)

### 11-2 hihello

In [0]:
import torch
import torch.optim as optim
import numpy as np

In [0]:
torch.manual_seed(0)

In [0]:
char_set = ['h', 'i', 'e', 'l', 'o']

In [0]:
#hyper parameters
input_size = len(char_set)
hidden_size = len(char_set)
learning_rate = 0.1

In [0]:
#data setting
x_data = [[0,1,0,2,3,3]] #input data니까 맨 마지막은 뺌
x_one_hot = [[[1,0,0,0,0],
             [0,1,0,0,0],
             [1,0,0,0,0],
             [0,0,1,0,0],
             [0,0,0,1,0],
             [0,0,0,1,0]]]
y_data = [[1,0,2,3,3,4]]

In [0]:
#transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [0]:
#declare RNN
rnn = torch.nn.RNN(input_size, hidden_size, batch_first = True)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)

In [0]:
for i in range(100):
  optimizer.zero_grad() #꼭 써줘야 한다! gradient를 초기화하는 역할, 안쓰면 gradient가 축적되어 잘못된 학습이 이뤄질수도 있다.
  outputs, _status = rnn(X) #_status: 다음 input이 있으면 rnn계산할 때 쓰이게 되는 hidden state

  loss = criterion(outputs.view(-1, input_size), Y.view(-1))
  loss.backward()
  optimizer.step()

  result = outputs.data.numpy().argmax(axis = 2)
  result_str = ''.join([char_set[c] for c in np.squeeze(result)])
  #np.squeeze: dimension이 1인 축을 없애주는 함수

  print(i, 'loss: ', loss.item(), 'prediction: ', result, 'true Y', y_data,
        'prediction str: ', result_str)

0 loss:  0.4556860029697418 prediction:  [[1 0 2 3 3 4]] true Y [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
1 loss:  0.4555518329143524 prediction:  [[1 0 2 3 3 4]] true Y [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
2 loss:  0.4554194211959839 prediction:  [[1 0 2 3 3 4]] true Y [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
3 loss:  0.45528873801231384 prediction:  [[1 0 2 3 3 4]] true Y [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
4 loss:  0.4551600515842438 prediction:  [[1 0 2 3 3 4]] true Y [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
5 loss:  0.4550330936908722 prediction:  [[1 0 2 3 3 4]] true Y [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
6 loss:  0.45490801334381104 prediction:  [[1 0 2 3 3 4]] true Y [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
7 loss:  0.454784631729126 prediction:  [[1 0 2 3 3 4]] true Y [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
8 loss:  0.4546630084514618 prediction:  [[1 0 2 3 3 4]] true Y [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
9 loss:  0.4545431

### 11_3 RNN Long sequence

In [0]:
import torch
import torch.optim as optim
import numpy as np

In [0]:
torch.manual_seed(0)

In [0]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [0]:
#dictionary 생성
char_set = list(set(sentence))
char_dic = {c:i for i, c in enumerate(char_set)}

In [0]:
#hyper parameter
dic_size = len(char_dic)
hidden_size = len(char_dic)
sequence_length = 10
learning_rate = 0.1 

In [0]:
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+sequence_length+1]
  print(i, x_str, '->', y_str)

  x_data.append([char_dic[c] for c in x_str])
  y_data.append([char_dic[c] for c in y_str])

x_one_hot = [np.eye(dic_size)[x] for x in x_data]

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [0]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [0]:
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers = layers,
                            batch_first = True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias = True)

  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)

    return x

net = Net(dic_size, hidden_size, 2)

In [0]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [0]:
for i in range(100):
  optimizer.zero_grad()
  outputs = net(X)
  loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
  loss.backward()
  optimizer.step()

  results = outputs.argmax(dim = 2)
  predict_str = ""

  for j, result in enumerate(results):
    #print(i, j, ''.join([char_set[t] for t in result]), loss.item())

    if j == 0:
      predict_str += ''.join([char_set[t] for t in result])
    
    else:
      predict_str += char_set[result[-1]]
  
  print(predict_str)

g tou want oo cuild anshim, don't arum up peoplemtogethem to collect word and don't ansign them tanks and dork, bui rathem toach them ta long for themtndless imm nsity of themtean
p tou want to build anship, don't arum up people eogethe  to bollect word and don't ansign the  tonks and dork, bui rathe  toach the  to long for the endless ipm nsity tf the eean
p tou want to build anship, don't drum up people eogethe  to bollect word and don't dnsign the  tonks and dork, bui rathe  toach the  to long for the endless ipm nsity of the eean
p tou want to build anship, don't drum up people eogethe  to bollect word and don't dnsign the  tanks and dork, bui rathe  toach the  ta long for the endless ipm nsity tf the eean
p tou want to build anship, don't arum up people eogethe  to bollect aood and don't ansign the  tonks and dook, bui rathe  toach the  to long for the endless ipm nsity tf the eean
p tou want to cuild anship, don't wrum up people eogethe  to collect word and don't wnsign the  tonk

### 11_4 RNN timeseries(시계열)

In [0]:
import torch
import torch.optim as optim
import numpy as nnp
import matplotlib.pyplot as plt

In [74]:
torch.manual_seed(0)

In [0]:
#scaling function for input data
#데이터마다 크기가 다르기 때문에 모두 0-1사이의 값으로 만들어주어 계산을 더 편하게 하기 위함

def minmax_scaler(data):
  numerator = data - np.min(data, 0)
  denominator = np.max(data, 0) - np,mean(data, 0)
  
  return numerator / (denominator + 1e-7)

In [0]:
#input을 위한 dataset
def build_dataset(time_series, seq_length):
  dataX = []
  dataY = []
  for i in range(0, len(time_series) - seq_length):
    _x = time_series[i : i+seq_length, :]
    _y = time_series[i + seq_length, [-1]]
    print(_x, '->', _y)
    dataX.append(_x)
    dataY.append(_y)
  
  return np.array(dataX), np.array(dataY)

In [0]:
#hyper parameters
seq_length = 7
data_dim = 5 #시가, 종가, 최고가 등 5개 dimension
hidden_dim = 10
output_dim = 1
learning_rate = 0.01
iterations = 500

In [78]:
xy = np.loadtxt('data-02-stock_daily.csv', delimiter = ',')
xy = xy[::-1]

# split train, test
train_size = int(len(xy) * 0.7)
train_set = xy[0:train_size]
test_set = xy[train_size - seq_length:]

train_set = minmax_scaler(train_set)
test_set = minmax_scaler(test_set)

trainX, trainY = build_dataset(train_set, seq_length)
testX, testY = build_dataset(test_set, seq_length)

trainX_tensor = torch.FloatTensor(trainX)
trainY_tensor = torch.FloatTensor(trainY)

testX_tensor = torch.FloatTensor(testX)
testY_tensor = torch.FloatTensor(testY)

OSError: ignored